In [ ]:
from espn_api.football import League
import numpy as np
import yaml
from random import shuffle
import time
from math import factorial
import pandas as pd
import matplotlib.pyplot as plt
from itertools import permutations

In [ ]:
# CONSTANTS TO SET!

ANALYSIS_YEAR = 2023
    
LEAGUE_ID = 111111111  # Your league ID (found in any URL related to your league)

NUM_SCHEDULE_SCENARIOS = 100000 # Set number of different schedule scenarios to try.

In [ ]:
# You'll need ESPN cookie info to get your league info.
# Instructions here: https://github.com/cwendt94/espn-api/discussions/150
# 
# I prefer to store this in a credentials yaml file,
# but you could also just paste them in here instead.

with open("credentials.yaml") as file:
    creds = yaml.load(file, Loader=yaml.FullLoader)

league = League(league_id=LEAGUE_ID, year=ANALYSIS_YEAR, espn_s2=creds["espn_s2"], swid=creds["swid"])

In [ ]:
# Test to make sure your league was imported properly
teams = league.teams
print(teams)

In [ ]:
# Get full schedule

full_schedule = []
for week, _ in enumerate(teams[0].schedule):
    if week < league.current_week:
        week_matchups = np.zeros((len(league.teams), len(league.teams)))
        for index, team in enumerate(teams):
            week_matchups[index][index] = 1
            week_matchups[index][team.schedule[week].team_id-1] = -1
        full_schedule.append(week_matchups)
    

In [ ]:
# Get scores from past weeks

scores = np.array([team.scores for team in league.teams])
print(scores[:,:])

In [ ]:
# Calculate wins

results = [(np.matmul(full_schedule[week], scores[:, week])*100).clip(min=0, max=1) for week in range(len(full_schedule)-1)]
# print(results)

In [ ]:
# Iterate through schedule scenarios

teams_list = league.teams

results = {
    team: {
        "min": league.current_week - 1,
        "max": 0,
        "avg": 0,
    }
for team in teams_list}

num_scenarios = NUM_SCHEDULE_SCENARIOS

# If the number of teams is small enough, we can look at all possible schedule setups explicitly!
if len(teams_list) <= 8:
    scenarios = list(permutations(teams_list))
    num_scenarios = len(scenarios)
    
start = time.time()
for i in range(num_scenarios):
    if len(teams_list) <= 8:
        teams_list = scenarios[i]
    else:
        # If there are more than 8 teams, we randomize the schedule scenarios
        # instead of trying to iterate through a giant list.
        shuffle(teams_list)
        
    scenario_scores = np.array([team.scores for team in teams_list])
    scenario_results = [
        (np.matmul(full_schedule[week],scenario_scores[:, week])*100)
        .clip(min=0, max=1)
        for week in range(len(full_schedule)-1)
    ]
    for index, team in enumerate(teams_list):
        wins = 0
        for w in scenario_results:
            wins = wins + w[index]
        if wins < results[team]["min"]:
            results[team]["min"] = wins
        if wins > results[team]["max"]:
            results[team]["max"] = wins
        results[team]["avg"] = results[team]["avg"] + wins

for team, data in results.items():
    data["avg"] = round(data["avg"] / num_scenarios, 2)
    data["actual"] = team.wins
    data["diff"] = round(data["actual"] - data["avg"], 2)

print("Time elapsed: " + str(time.time()-start))
print("Scenarios calculated: " + str(num_scenarios))

In [ ]:
# Easy-to-read results
friendly_results = {team.team_name:data for (team, data) in results.items()}
results_df = pd.DataFrame.from_dict(friendly_results, orient="index")
results_df.sort_values("avg", inplace=True, ascending=False)
display(results_df)

In [ ]:
# Schedule luck visualization

teams = results_df.index
schedule_luck = results_df["diff"]
actual_wins = results_df["actual"]
max_wins = results_df["max"]
min_wins = results_df["min"]
expected_wins = results_df["avg"]

fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_axes([0, 0, 1, 1])
bars = ax.bar(teams, schedule_luck)

plt.xticks(rotation=90)
for bar in bars:
    x = bar.get_x()
    y = bar.get_height()
    width = bar.get_width()
    offset = 0.02
    if y < 0:
        y_pos = y - (offset + 0.05)
        value = f"{y}"
    else:
        y_pos = y + offset
        value = f"+{y}"
    plt.text(x + width/2, y_pos, value, weight="bold", ha="center", color=("red" if y < 0 else "black"))

    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)
plt.title("Actual Wins Minus Expected Wins", fontsize=18)


In [ ]:
# Season summary visualization

fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_axes([0, 0, 1, 0.9])

week_list = [league.current_week - 1] * len(teams_list)

expected_points, = ax.plot(teams, expected_wins, ls="", marker=".", ms=30, label="Expected Wins")
max_points, = ax.plot(teams, max_wins, ls="", color="green", marker=".", ms=30, label="Max Possible Wins")
min_points, = ax.plot(teams, min_wins, ls="", color="red", marker=".", ms=30, label="Min Possible Wins")
actual_points, = ax.plot(teams, actual_wins, ls="", color="black", marker="x", markeredgewidth=5, ms=20, label="Actual Wins")

range_lines = ax.vlines(teams, min_wins, max_wins, color="#cccccc", ls="dashed")



plt.xticks(rotation=90)

    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(bottom=False, left=False)
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)
ax.legend(
    handles=[expected_points, actual_points, max_points, min_points],
    loc="upper center",
    bbox_to_anchor=(0, 1, 1, .1),
    mode="expand",
    ncol=4,
    borderpad=1.2
)